In [1]:
library(readr)
library(dplyr)
library(tidyr)
library(stats)
library(dplyr)
# For Tukey's HSD test
library(multcomp)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: mvtnorm

Loading required package: survival

Loading required package: TH.data

Loading required package: MASS


Attaching package: 'MASS'


The following object is masked from 'package:dplyr':

    select



Attaching package: 'TH.data'


The following object is masked from 'package:MASS':

    geyser




In [2]:
# Load the CSV file
survey_data <- read.csv("../data/raw/PSYC 421 - Single-Use Cup_March 12, 2024_16.06.csv", stringsAsFactors = FALSE)

# View the first few rows of the data frame
head(survey_data)

,StartDate,EndDate,Status,IPAddress,Progress,Duration..in.seconds.,Finished,RecordedDate,ResponseId,RecipientLastName,...,Q3.2,Q4.2,Q5,Q5_5_TEXT,Q6,Q6_5_TEXT,Q7,Q8,Q8_4_TEXT,ConditionAssigned
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,How likely are you to reduce using single use cups?,How likely are you to bring your own reusable cup when purchasing beverages?,What is your status at UBC? - Selected Choice,What is your status at UBC? - Other (please specify) - Text,What is your gender identity? - Selected Choice,What is your gender identity? - Other (please specify) - Text,What is your age? (in number of years),What factor(s) would most likely reduce your consumption of single-use cups? (Choose all applicable answers) - Selected Choice,What factor(s) would most likely reduce your consumption of single-use cups? (Choose all applicable answers) - Other (Specify) - Text,ConditionAssigned
2,"{""ImportId"":""startDate"",""timeZone"":""America/Denver""}","{""ImportId"":""endDate"",""timeZone"":""America/Denver""}","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America/Denver""}","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID52""}","{""ImportId"":""QID53""}","{""ImportId"":""QID22""}","{""ImportId"":""QID22_5_TEXT""}","{""ImportId"":""QID24""}","{""ImportId"":""QID24_5_TEXT""}","{""ImportId"":""QID23_TEXT""}","{""ImportId"":""QID32""}","{""ImportId"":""QID32_4_TEXT""}","{""ImportId"":""ConditionAssigned""}"
3,2024-03-06 14:25:55,2024-03-06 14:27:13,IP Address,206.12.179.98,100,77,True,2024-03-06 14:27:13,R_7P4M3QNUV0sz7kl,,...,,,Students,,Woman,,21,"Learning about the environmental consequences associated with single-use cups (i.e,. pollution, cups used per day),Preferences for alternatives of single-use cup (i.e., I prefer to use my own mug)",,Loss
4,2024-03-06 14:36:46,2024-03-06 14:37:20,IP Address,24.84.220.242,100,34,True,2024-03-06 14:37:21,R_3uPoKeQUqogIL4l,,...,,,Not Affiliated,,Man,,21,"Learning about the financial implications of using single-use cups (i.e., cost per cup)",,Gain
5,2024-03-06 15:59:14,2024-03-06 16:00:22,IP Address,108.180.201.174,100,68,True,2024-03-06 16:00:23,R_65G7BrispC2eV2n,,...,Slightly unlikely,Slightly unlikely,Students,,Woman,,22,"Learning about the financial implications of using single-use cups (i.e., cost per cup),Preferences for alternatives of single-use cup (i.e., I prefer to use my own mug)",,Control
6,2024-03-06 17:01:22,2024-03-06 17:04:00,IP Address,49.36.185.105,100,158,True,2024-03-06 17:04:01,R_40i6X1z7fQUH2Bs,,...,,,Not Affiliated,,Woman,,,"Preferences for alternatives of single-use cup (i.e., I prefer to use my own mug)",,Loss


In [3]:
# Get the columns in the survey
column_names <- names(survey_data)
column_names


[1] "StartDate"             "EndDate"               "Status"               
 [4] "IPAddress"             "Progress"              "Duration..in.seconds."
 [7] "Finished"              "RecordedDate"          "ResponseId"           
[10] "RecipientLastName"     "RecipientFirstName"    "RecipientEmail"       
[13] "ExternalReference"     "LocationLatitude"      "LocationLongitude"    
[16] "DistributionChannel"   "UserLanguage"          "Consent.Form"         
[19] "Q2"                    "Q3"                    "Q4"                   
[22] "Q2.1"                  "Q3.1"                  "Q4.1"                 
[25] "Q2.2"                  "Q3.2"                  "Q4.2"                 
[28] "Q5"                    "Q5_5_TEXT"             "Q6"                   
[31] "Q6_5_TEXT"             "Q7"                    "Q8"                   
[34] "Q8_4_TEXT"             "ConditionAssigned"

In [4]:
filter_df <- dplyr::select(survey_data, 
  `Duration..in.seconds.`, `Finished`, `ResponseId`, 
  `Q2`, `Q3`, `Q4`, 
  `Q2.1`, `Q3.1`, `Q4.1`, 
  `Q2.2`, `Q3.2`, `Q4.2`, 
  `Q5`, `Q5_5_TEXT`, `Q6`, `Q6_5_TEXT`, `Q7`, `Q8`, `Q8_4_TEXT`, 
  `ConditionAssigned`
)


In [5]:
# Remove the second row as this additional data is not needed
df_cleaned <- filter_df[-2, ]
# View the first few rows to confirm removal
head(df_cleaned)

,Duration..in.seconds.,Finished,ResponseId,Q2,Q3,Q4,Q2.1,Q3.1,Q4.1,Q2.2,Q3.2,Q4.2,Q5,Q5_5_TEXT,Q6,Q6_5_TEXT,Q7,Q8,Q8_4_TEXT,ConditionAssigned
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Duration (in seconds),Finished,Response ID,How surprising was the above information?,How likely are you to reduce using single use cups?,How likely are you to bring your own reusable cup when purchasing beverages?,How surprising was the above information?,How likely are you to reduce using single use cups?,How likely are you to bring your own reusable cup when purchasing beverages?,How surprising was the above information?,How likely are you to reduce using single use cups?,How likely are you to bring your own reusable cup when purchasing beverages?,What is your status at UBC? - Selected Choice,What is your status at UBC? - Other (please specify) - Text,What is your gender identity? - Selected Choice,What is your gender identity? - Other (please specify) - Text,What is your age? (in number of years),What factor(s) would most likely reduce your consumption of single-use cups? (Choose all applicable answers) - Selected Choice,What factor(s) would most likely reduce your consumption of single-use cups? (Choose all applicable answers) - Other (Specify) - Text,ConditionAssigned
3,77,True,R_7P4M3QNUV0sz7kl,,,,Extremely unsurprising,Slightly unlikely,Slightly likely,,,,Students,,Woman,,21,"Learning about the environmental consequences associated with single-use cups (i.e,. pollution, cups used per day),Preferences for alternatives of single-use cup (i.e., I prefer to use my own mug)",,Loss
4,34,True,R_3uPoKeQUqogIL4l,Slightly surprising,Moderately unlikely,Moderately unlikely,,,,,,,Not Affiliated,,Man,,21,"Learning about the financial implications of using single-use cups (i.e., cost per cup)",,Gain
5,68,True,R_65G7BrispC2eV2n,,,,,,,Slightly surprising,Slightly unlikely,Slightly unlikely,Students,,Woman,,22,"Learning about the financial implications of using single-use cups (i.e., cost per cup),Preferences for alternatives of single-use cup (i.e., I prefer to use my own mug)",,Control
6,158,True,R_40i6X1z7fQUH2Bs,,,,Moderately unsurprising,Moderately likely,Slightly likely,,,,Not Affiliated,,Woman,,,"Preferences for alternatives of single-use cup (i.e., I prefer to use my own mug)",,Loss
7,91,True,R_1k0T1HeuJsmtdQm,Slightly surprising,Slightly unlikely,Slightly unlikely,,,,,,,Students,,Man,,21,"Learning about the financial implications of using single-use cups (i.e., cost per cup)",,Gain


In [6]:
# Specify the path and filename and add to processed data
output_path <- "../data/processed/df_cleaned.csv"

# Write the dataframe to a CSV file
write.csv(df_cleaned, output_path, row.names = FALSE)


In [7]:
# Load the CSV file
survey_data_numeric <- read.csv("../data/raw/PSYC 421 - Single-Use Cup_March 12, 2024_17.02_numeric.csv", stringsAsFactors = FALSE)

# View the first few rows of the data frame
head(survey_data_numeric)

,StartDate,EndDate,Status,IPAddress,Progress,Duration..in.seconds.,Finished,RecordedDate,ResponseId,RecipientLastName,...,Q3.2,Q4.2,Q5,Q5_5_TEXT,Q6,Q6_5_TEXT,Q7,Q8,Q8_4_TEXT,ConditionAssigned
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,How likely are you to reduce using single use cups?,How likely are you to bring your own reusable cup when purchasing beverages?,What is your status at UBC? - Selected Choice,What is your status at UBC? - Other (please specify) - Text,What is your gender identity? - Selected Choice,What is your gender identity? - Other (please specify) - Text,What is your age? (in number of years),What factor(s) would most likely reduce your consumption of single-use cups? (Choose all applicable answers) - Selected Choice,What factor(s) would most likely reduce your consumption of single-use cups? (Choose all applicable answers) - Other (Specify) - Text,ConditionAssigned
2,"{""ImportId"":""startDate"",""timeZone"":""America/Denver""}","{""ImportId"":""endDate"",""timeZone"":""America/Denver""}","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America/Denver""}","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID52""}","{""ImportId"":""QID53""}","{""ImportId"":""QID22""}","{""ImportId"":""QID22_5_TEXT""}","{""ImportId"":""QID24""}","{""ImportId"":""QID24_5_TEXT""}","{""ImportId"":""QID23_TEXT""}","{""ImportId"":""QID32""}","{""ImportId"":""QID32_4_TEXT""}","{""ImportId"":""ConditionAssigned""}"
3,2024-03-06 14:25:55,2024-03-06 14:27:13,0,206.12.179.98,100,77,1,2024-03-06 14:27:13,R_7P4M3QNUV0sz7kl,,...,,,1,,2,,21,"1,3",,Loss
4,2024-03-06 14:36:46,2024-03-06 14:37:20,0,24.84.220.242,100,34,1,2024-03-06 14:37:21,R_3uPoKeQUqogIL4l,,...,,,4,,1,,21,2,,Gain
5,2024-03-06 15:59:14,2024-03-06 16:00:22,0,108.180.201.174,100,68,1,2024-03-06 16:00:23,R_65G7BrispC2eV2n,,...,3,3,1,,2,,22,"2,3",,Control
6,2024-03-06 17:01:22,2024-03-06 17:04:00,0,49.36.185.105,100,158,1,2024-03-06 17:04:01,R_40i6X1z7fQUH2Bs,,...,,,4,,2,,,3,,Loss


In [8]:
# keep only selected columns for analysis

filter_df_numerical <- dplyr::select(survey_data_numeric, 
  `Duration..in.seconds.`, `Finished`, `ResponseId`, 
  `Q2`, `Q3`, `Q4`, 
  `Q2.1`, `Q3.1`, `Q4.1`, 
  `Q2.2`, `Q3.2`, `Q4.2`, 
  `Q5`, `Q5_5_TEXT`, `Q6`, `Q6_5_TEXT`, `Q7`, `Q8`, `Q8_4_TEXT`, 
  `ConditionAssigned`
)
# View the first few rows of the modified dataframe
head(filter_df_numerical)


,Duration..in.seconds.,Finished,ResponseId,Q2,Q3,Q4,Q2.1,Q3.1,Q4.1,Q2.2,Q3.2,Q4.2,Q5,Q5_5_TEXT,Q6,Q6_5_TEXT,Q7,Q8,Q8_4_TEXT,ConditionAssigned
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Duration (in seconds),Finished,Response ID,How surprising was the above information?,How likely are you to reduce using single use cups?,How likely are you to bring your own reusable cup when purchasing beverages?,How surprising was the above information?,How likely are you to reduce using single use cups?,How likely are you to bring your own reusable cup when purchasing beverages?,How surprising was the above information?,How likely are you to reduce using single use cups?,How likely are you to bring your own reusable cup when purchasing beverages?,What is your status at UBC? - Selected Choice,What is your status at UBC? - Other (please specify) - Text,What is your gender identity? - Selected Choice,What is your gender identity? - Other (please specify) - Text,What is your age? (in number of years),What factor(s) would most likely reduce your consumption of single-use cups? (Choose all applicable answers) - Selected Choice,What factor(s) would most likely reduce your consumption of single-use cups? (Choose all applicable answers) - Other (Specify) - Text,ConditionAssigned
2,"{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""_recordId""}","{""ImportId"":""QID31""}","{""ImportId"":""QID39""}","{""ImportId"":""QID29""}","{""ImportId"":""QID48""}","{""ImportId"":""QID50""}","{""ImportId"":""QID51""}","{""ImportId"":""QID49""}","{""ImportId"":""QID52""}","{""ImportId"":""QID53""}","{""ImportId"":""QID22""}","{""ImportId"":""QID22_5_TEXT""}","{""ImportId"":""QID24""}","{""ImportId"":""QID24_5_TEXT""}","{""ImportId"":""QID23_TEXT""}","{""ImportId"":""QID32""}","{""ImportId"":""QID32_4_TEXT""}","{""ImportId"":""ConditionAssigned""}"
3,77,1,R_7P4M3QNUV0sz7kl,,,,1,3,5,,,,1,,2,,21,"1,3",,Loss
4,34,1,R_3uPoKeQUqogIL4l,5,2,2,,,,,,,4,,1,,21,2,,Gain
5,68,1,R_65G7BrispC2eV2n,,,,,,,5,3,3,1,,2,,22,"2,3",,Control
6,158,1,R_40i6X1z7fQUH2Bs,,,,2,6,5,,,,4,,2,,,3,,Loss


In [9]:
# Remove the second row as this additional data is not needed
df_cleaned_numerical <- filter_df_numerical[-2, ]

# View the first few rows to confirm removal
head(df_cleaned_numerical)

,Duration..in.seconds.,Finished,ResponseId,Q2,Q3,Q4,Q2.1,Q3.1,Q4.1,Q2.2,Q3.2,Q4.2,Q5,Q5_5_TEXT,Q6,Q6_5_TEXT,Q7,Q8,Q8_4_TEXT,ConditionAssigned
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Duration (in seconds),Finished,Response ID,How surprising was the above information?,How likely are you to reduce using single use cups?,How likely are you to bring your own reusable cup when purchasing beverages?,How surprising was the above information?,How likely are you to reduce using single use cups?,How likely are you to bring your own reusable cup when purchasing beverages?,How surprising was the above information?,How likely are you to reduce using single use cups?,How likely are you to bring your own reusable cup when purchasing beverages?,What is your status at UBC? - Selected Choice,What is your status at UBC? - Other (please specify) - Text,What is your gender identity? - Selected Choice,What is your gender identity? - Other (please specify) - Text,What is your age? (in number of years),What factor(s) would most likely reduce your consumption of single-use cups? (Choose all applicable answers) - Selected Choice,What factor(s) would most likely reduce your consumption of single-use cups? (Choose all applicable answers) - Other (Specify) - Text,ConditionAssigned
3,77,1,R_7P4M3QNUV0sz7kl,,,,1,3,5,,,,1,,2,,21,"1,3",,Loss
4,34,1,R_3uPoKeQUqogIL4l,5,2,2,,,,,,,4,,1,,21,2,,Gain
5,68,1,R_65G7BrispC2eV2n,,,,,,,5,3,3,1,,2,,22,"2,3",,Control
6,158,1,R_40i6X1z7fQUH2Bs,,,,2,6,5,,,,4,,2,,,3,,Loss
7,91,1,R_1k0T1HeuJsmtdQm,5,3,3,,,,,,,1,,1,,21,2,,Gain


In [10]:
# Specify the path and filename and add to processed data
output_path <- "../data/processed/df_cleaned_numerical.csv"

# Write the dataframe to a CSV file
write.csv(df_cleaned_numerical, output_path, row.names = FALSE)


In [11]:
df_cleaned_numerical <- df_cleaned_numerical[-1, ]


In [12]:

# Convert Q2 Q3 and Q4 columns to numeric
df_cleaned_numerical$Q2 <- as.numeric(as.character(df_cleaned_numerical$Q2))
df_cleaned_numerical$`Q2.1` <- as.numeric(as.character(df_cleaned_numerical$`Q2.1`))
df_cleaned_numerical$`Q2.2` <- as.numeric(as.character(df_cleaned_numerical$`Q2.2`))

df_cleaned_numerical$Q3 <- as.numeric(as.character(df_cleaned_numerical$Q3))
df_cleaned_numerical$`Q3.1` <- as.numeric(as.character(df_cleaned_numerical$`Q3.1`))
df_cleaned_numerical$`Q3.2` <- as.numeric(as.character(df_cleaned_numerical$`Q3.2`))

df_cleaned_numerical$Q4 <- as.numeric(as.character(df_cleaned_numerical$Q4))
df_cleaned_numerical$`Q4.1` <- as.numeric(as.character(df_cleaned_numerical$`Q4.1`))
df_cleaned_numerical$`Q4.2` <- as.numeric(as.character(df_cleaned_numerical$`Q4.2`))

# Create Q2_combined Q3_combined and Q4_combined columns
df_cleaned_numerical <- df_cleaned_numerical %>%
  mutate(
    Q2_combined = case_when(
      ConditionAssigned == "Gain" ~ Q2,
      ConditionAssigned == "Loss" ~ `Q2.1`,
      ConditionAssigned == "Control" ~ `Q2.2`,
      TRUE ~ NA_real_
    ),
    Q3_combined = case_when(
      ConditionAssigned == "Gain" ~ Q3,
      ConditionAssigned == "Loss" ~ `Q3.1`,
      ConditionAssigned == "Control" ~ `Q3.2`,
      TRUE ~ NA_real_
    ),
    Q4_combined = case_when(
      ConditionAssigned == "Gain" ~ Q4,
      ConditionAssigned == "Loss" ~ `Q4.1`,
      ConditionAssigned == "Control" ~ `Q4.2`,
      TRUE ~ NA_real_
    )
  )

# Remove the original Q2 Q3 and Q4 columns
df_cleaned_numerical <- df_cleaned_numerical %>%
  dplyr::select(-Q2, -`Q2.1`, -`Q2.2`,-Q3, -`Q3.1`, -`Q3.2`, -Q4, -`Q4.1`, -`Q4.2`, everything())

# View the updated dataframe
head(df_cleaned_numerical)


,Duration..in.seconds.,Finished,ResponseId,Q5,Q5_5_TEXT,Q6,Q6_5_TEXT,Q7,Q8,Q8_4_TEXT,...,Q4_combined,Q2,Q3,Q4,Q2.1,Q3.1,Q4.1,Q2.2,Q3.2,Q4.2
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,77,1,R_7P4M3QNUV0sz7kl,1,,2,,21,"1,3",,...,5,NA,NA,NA,1,3,5,NA,NA,NA
4,34,1,R_3uPoKeQUqogIL4l,4,,1,,21,2,,...,2,5,2,2,NA,NA,NA,NA,NA,NA
5,68,1,R_65G7BrispC2eV2n,1,,2,,22,"2,3",,...,3,NA,NA,NA,NA,NA,NA,5,3,3
6,158,1,R_40i6X1z7fQUH2Bs,4,,2,,,3,,...,5,NA,NA,NA,2,6,5,NA,NA,NA
7,91,1,R_1k0T1HeuJsmtdQm,1,,1,,21,2,,...,3,5,3,3,NA,NA,NA,NA,NA,NA
8,103,1,R_3y9bl3tR1vzxcSR,1,,2,,22,"1,2",,...,6,NA,NA,NA,2,7,6,NA,NA,NA


In [13]:
names(df_cleaned_numerical)

[1] "Duration..in.seconds." "Finished"              "ResponseId"           
 [4] "Q5"                    "Q5_5_TEXT"             "Q6"                   
 [7] "Q6_5_TEXT"             "Q7"                    "Q8"                   
[10] "Q8_4_TEXT"             "ConditionAssigned"     "Q2_combined"          
[13] "Q3_combined"           "Q4_combined"           "Q2"                   
[16] "Q3"                    "Q4"                    "Q2.1"                 
[19] "Q3.1"                  "Q4.1"                  "Q2.2"                 
[22] "Q3.2"                  "Q4.2"

In [14]:
df_cleaned_numerical_final <- dplyr::select(df_cleaned_numerical, 
  `ResponseId`, 
  `ConditionAssigned`,
  `Q2_combined`,`Q3_combined`, `Q4_combined`
  #`Q2`, `Q2.1`, `Q2.2`,`Q3`, `Q3.1`,`Q3.2`, `Q4`,`Q4.1`,`Q4.2`
)
df_cleaned_numerical_final

,ResponseId,ConditionAssigned,Q2_combined,Q3_combined,Q4_combined
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
3,R_7P4M3QNUV0sz7kl,Loss,1,3,5
4,R_3uPoKeQUqogIL4l,Gain,5,2,2
5,R_65G7BrispC2eV2n,Control,5,3,3
6,R_40i6X1z7fQUH2Bs,Loss,2,6,5
7,R_1k0T1HeuJsmtdQm,Gain,5,3,3
8,R_3y9bl3tR1vzxcSR,Loss,2,7,6
9,R_7jx25gH6q84XuJX,Control,1,5,4
10,R_1F5my25GorP5SB7,Control,1,3,3
11,R_4rrxy04vs26yGbE,Gain,5,5,3


In [15]:
# Specify the path and filename and add to processed data
output_path <- "../data/processed/numerical_final.csv"

# Write the dataframe to a CSV file
write.csv(df_cleaned_numerical_final, output_path, row.names = FALSE)

## **One-Way ANOVA**

We conducted an ANOVA (Analysis of Variance) to examine if there were statistically significant differences between the means of `Q2_combined`, `Q3_combined`, and `Q4_combined` scores across the different `ConditionAssigned` groups. Below is a summary of the findings from each ANOVA test:

### For `Q2_combined`: How surprising was the above information? 

- **Degrees of Freedom (Df)**: The analysis included 2 degrees of freedom for `ConditionAssigned` (reflecting the 3 conditions) and 65 degrees of freedom for residuals (errors).
- **Sum of Squares (Sum Sq)**: The variability was 23.45 for the conditions and 183.61 for the residuals, indicating the spread of scores within and between conditions.
- **Mean Square (Mean Sq)**: This value, calculated as the sum of squares divided by the respective degrees of freedom, was used in determining the F value. The mean squares were 11.723 for conditions and 2.825 for residuals.
- **F value**: The F statistic was 4.15, indicating the ratio of the variance between groups to the variance within groups.
- **P-value (Pr(>F))**: The p-value was 0.0201, which is below the 0.05 threshold for statistical significance, suggesting significant differences in `Q2_combined` scores across the conditions.

### For `Q3_combined`: How likely are you to reduce using single use cups?

- The p-value was 0.261, above the 0.05 threshold, indicating no statistically significant differences in `Q3_combined` scores across the conditions.

### For `Q4_combined`: 

- Similar to `Q2_combined`, the `Q4_combined` scores yielded a p-value of 0.00939, below the 0.05 threshold, indicating statistically significant differences across conditions.

### Summary: How likely are you to bring your own reusable cup when purchasing beverages?

- The ANOVA tests revealed statistically significant differences in the `Q2_combined` and `Q4_combined` scores across conditions, suggesting that the assigned condition likely influenced participants' responses to these questions.
- The `Q3_combined` scores did not exhibit statistically significant differences across conditions, indicating that the assigned condition did not significantly influence responses to this particular question.
- The significance codes (`*` for p < 0.05, `**` for p < 0.01) highlight which comparisons were statistically significant. Both `Q2_combined` and `Q4_combined` were significant at the p < 0.05 level, suggesting a strong likelihood that the observed differences are not due to random chance.
- A note on data handling: Observations were deleted due to missingness. It's crucial to consider how missing data may impact the analysis and to ensure that the results are not biased by the way missing data are treated.

In conclusion, the ANOVA analysis indicates that the condition participants were assigned to had a significant impact on their responses to certain survey questions (`Q2_combined` and `Q4_combined`), while it did not significantly affect responses to other questions (`Q3_combined`).

In [16]:
# For Q2_combined
aov_Q2 <- aov(Q2_combined ~ ConditionAssigned, data = df_cleaned_numerical)
summary(aov_Q2)

# For Q3_combined
aov_Q3 <- aov(Q3_combined ~ ConditionAssigned, data = df_cleaned_numerical)
summary(aov_Q3)

# For Q4_combined
aov_Q4 <- aov(Q4_combined ~ ConditionAssigned, data = df_cleaned_numerical)
summary(aov_Q4)


                  Df Sum Sq Mean Sq F value Pr(>F)  
ConditionAssigned  2  23.45  11.723    4.15 0.0201 *
Residuals         65 183.61   2.825                 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
1 observation deleted due to missingness

                  Df Sum Sq Mean Sq F value Pr(>F)
ConditionAssigned  2   8.99   4.495   1.371  0.261
Residuals         64 209.88   3.279               
2 observations deleted due to missingness

                  Df Sum Sq Mean Sq F value  Pr(>F)   
ConditionAssigned  2  28.07  14.035    5.02 0.00939 **
Residuals         65 181.74   2.796                   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
1 observation deleted due to missingness

In [17]:
# Normality
shapiro.test(df_cleaned_numerical$Q2_combined)

# Homogeneity of variance
bartlett.test(Q2_combined ~ ConditionAssigned, data = df_cleaned_numerical)



	Shapiro-Wilk normality test

data:  df_cleaned_numerical$Q2_combined
W = 0.88352, p-value = 1.192e-05



	Bartlett test of homogeneity of variances

data:  Q2_combined by ConditionAssigned
Bartlett's K-squared = 0.2581, df = 2, p-value = 0.8789


## **Tukey's HSD Test**

### How surprising was the above information? 


The Tukey's Honest Significant Difference (HSD) test results for `Q2_combined` scores across different conditions ("Gain", "Loss", and "Control") provide detailed pairwise comparisons between these groups. Here's what the results indicate:

### Pairwise Comparisons:

1. **Gain vs. Control**:
   - **Difference**: The mean difference between the "Gain" and "Control" groups is 1.332016.
   - **Confidence Interval**: The 95% confidence interval for this difference ranges from 0.12982247 to 2.534209.
   - **Adjusted P-value**: The p-value for this comparison is 0.0264126, which is less than 0.05, indicating a statistically significant difference between the "Gain" and "Control" conditions. This suggests that the "Gain" condition led to higher `Q2_combined` scores compared to the "Control" condition.

2. **Loss vs. Control**:
   - **Difference**: The mean difference between the "Loss" and "Control" groups is 1.130435.
   - **Confidence Interval**: The 95% confidence interval for this difference ranges from -0.05832581 to 2.319195.
   - **Adjusted P-value**: The p-value for this comparison is 0.0656822, which is slightly above the 0.05 threshold, indicating that the difference between the "Loss" and "Control" conditions is not statistically significant at the 5% level. However, it's close to significance, suggesting a trend where the "Loss" condition may lead to higher `Q2_combined` scores compared to the "Control" condition, but this finding isn't as robust.

3. **Loss vs. Gain**:
   - **Difference**: The mean difference between the "Loss" and "Gain" groups is -0.201581.
   - **Confidence Interval**: The 95% confidence interval for this difference ranges from -1.40377437 to 1.000612.
   - **Adjusted P-value**: The p-value for this comparison is 0.9148228, which is much greater than 0.05, indicating no statistically significant difference between the "Loss" and "Gain" conditions. This suggests that there is no significant difference in `Q2_combined` scores between these two conditions.

### Summary:

The Tukey HSD test results suggest that participants in the "Gain" condition had significantly higher `Q2_combined` scores compared to those in the "Control" condition. There was no statistically significant difference between the "Loss" and "Gain" conditions, nor between the "Loss" and "Control" conditions, although the "Loss" vs. "Control" comparison was close to significance. This analysis provides a more nuanced understanding of how different conditions influenced the `Q2_combined` scores, highlighting the specific conditions that led to significant differences.

In [18]:
# Tukey HSD test for Q2_combined
tukey_Q2 <- TukeyHSD(aov(Q2_combined ~ ConditionAssigned, data = df_cleaned_numerical))
print(tukey_Q2)

  Tukey multiple comparisons of means
    95% family-wise confidence level

Fit: aov(formula = Q2_combined ~ ConditionAssigned, data = df_cleaned_numerical)

$ConditionAssigned
                  diff         lwr      upr     p adj
Gain-Control  1.332016  0.12982247 2.534209 0.0264126
Loss-Control  1.130435 -0.05832581 2.319195 0.0656822
Loss-Gain    -0.201581 -1.40377437 1.000612 0.9148228



### How likely are you to reduce using single use cups?

The Tukey's Honest Significant Difference (HSD) test results for `Q3_combined` scores, which represent how likely participants are to reduce using single-use cups, provide pairwise comparisons between the different conditions ("Gain", "Loss", and "Control"). Here's the interpretation of these results:

### Pairwise Comparisons:

1. **Gain vs. Control**:
   - **Difference**: The mean difference between the "Gain" and "Control" groups is 0.5776398.
   - **Confidence Interval**: The 95% confidence interval for this difference ranges from -0.7338076 to 1.889087.
   - **Adjusted P-value**: The p-value for this comparison is 0.5441534, indicating no statistically significant difference between the "Gain" and "Control" conditions. This suggests that the likelihood of participants reducing the use of single-use cups does not significantly differ between these two conditions.

2. **Loss vs. Control**:
   - **Difference**: The mean difference between the "Loss" and "Control" groups is 0.8695652.
   - **Confidence Interval**: The 95% confidence interval for this difference ranges from -0.4117298 to 2.150860.
   - **Adjusted P-value**: The p-value for this comparison is 0.2412153, also indicating no statistically significant difference between the "Loss" and "Control" conditions. This result suggests that the condition does not significantly affect participants' likelihood to reduce using single-use cups compared to the control condition.

3. **Loss vs. Gain**:
   - **Difference**: The mean difference between the "Loss" and "Gain" groups is 0.2919255.
   - **Confidence Interval**: The 95% confidence interval for this difference ranges from -1.0195218 to 1.603373.
   - **Adjusted P-value**: The p-value for this comparison is 0.8548785, indicating no statistically significant difference between the "Loss" and "Gain" conditions. This suggests that the two intervention conditions do not differ significantly in their effect on participants' likelihood to reduce the use of single-use cups.

### Summary:

The Tukey HSD test results for the `Q3_combined` scores indicate that there are no statistically significant differences between any of the pairwise comparisons among the "Gain", "Loss", and "Control" conditions. This implies that the intervention conditions did not significantly influence participants' reported likelihood of reducing the use of single-use cups. All comparisons show a lack of statistical significance, suggesting that any differences observed in the ANOVA for `Q3_combined` scores do not translate into specific differences between the groups when examined closely with post-hoc analysis.

In [19]:
# Tukey HSD test for Q2_combined
tukey_Q3 <- TukeyHSD(aov(Q3_combined ~ ConditionAssigned, data = df_cleaned_numerical))
print(tukey_Q3)

  Tukey multiple comparisons of means
    95% family-wise confidence level

Fit: aov(formula = Q3_combined ~ ConditionAssigned, data = df_cleaned_numerical)

$ConditionAssigned
                  diff        lwr      upr     p adj
Gain-Control 0.5776398 -0.7338076 1.889087 0.5441534
Loss-Control 0.8695652 -0.4117298 2.150860 0.2412153
Loss-Gain    0.2919255 -1.0195218 1.603373 0.8548785



### How likely are you to bring your own reusable cup when purchasing beverages?

The Tukey's Honest Significant Difference (HSD) test results for `Q4_combined` scores, related to how likely participants are to bring their own reusable cup when purchasing beverages, show pairwise comparisons among different conditions ("Gain", "Loss", and "Control"). Here's the interpretation of these results:

### Pairwise Comparisons:

1. **Gain vs. Control**:
   - **Difference**: The mean difference between the "Gain" and "Control" groups is 0.173913.
   - **Confidence Interval**: The 95% confidence interval ranges from -1.02213120 to 1.369957.
   - **Adjusted P-value**: The p-value for this comparison is 0.9352181, indicating no statistically significant difference between the "Gain" and "Control" conditions. This suggests that the likelihood of participants bringing their own reusable cups does not significantly differ between these two conditions.

2. **Loss vs. Control**:
   - **Difference**: The mean difference between the "Loss" and "Control" groups is 1.434783.
   - **Confidence Interval**: The 95% confidence interval ranges from 0.25210241 to 2.617463.
   - **Adjusted P-value**: The p-value for this comparison is 0.0135341, indicating a statistically significant difference between the "Loss" and "Control" conditions. This result suggests that participants in the "Loss" condition are significantly more likely to bring their own reusable cups compared to those in the "Control" condition.

3. **Loss vs. Gain**:
   - **Difference**: The mean difference between the "Loss" and "Gain" groups is 1.260870.
   - **Confidence Interval**: The 95% confidence interval ranges from 0.06482533 to 2.456914.
   - **Adjusted P-value**: The p-value for this comparison is 0.0365273, indicating a statistically significant difference between the "Loss" and "Gain" conditions. This suggests that participants in the "Loss" condition are significantly more likely to bring their own reusable cups compared to those in the "Gain" condition.

### Summary:

The Tukey HSD test results for the `Q4_combined` scores indicate statistically significant differences between the "Loss" condition and both the "Control" and "Gain" conditions. Specifically, participants in the "Loss" condition reported a higher likelihood of bringing their own reusable cups when purchasing beverages compared to those in either the "Control" or "Gain" conditions. There was no statistically significant difference between the "Gain" and "Control" conditions, suggesting that the intervention did not significantly affect participants' likelihood to bring their own reusable cups in these two groups compared to each other. 

These findings provide valuable insights into how different types of interventions might influence environmentally conscious behaviors, such as the use of reusable cups, highlighting the effectiveness of the "Loss" condition in promoting such behavior.

In [20]:
# Tukey HSD test for Q4_combined
tukey_Q4 <- TukeyHSD(aov(Q4_combined ~ ConditionAssigned, data = df_cleaned_numerical))
print(tukey_Q4)

  Tukey multiple comparisons of means
    95% family-wise confidence level

Fit: aov(formula = Q4_combined ~ ConditionAssigned, data = df_cleaned_numerical)

$ConditionAssigned
                 diff         lwr      upr     p adj
Gain-Control 0.173913 -1.02213120 1.369957 0.9352181
Loss-Control 1.434783  0.25210241 2.617463 0.0135341
Loss-Gain    1.260870  0.06482533 2.456914 0.0365273

